<a href="https://colab.research.google.com/github/RoshanTS/pfe-estia/blob/extract_twitter_script/extraction_donn%C3%A9es_via_api_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script d'extraction de données twitter

##Importations of Python modules

We import the following libraries :

*   [tweepy](https://docs.tweepy.org/en/latest/) :  python client for the official [Twitter API](https://developer.twitter.com/en.html) 
*   [textblob](https://docs.tweepy.org/en/latest/) : python library for processing textual data
*  [os](https://docs.python.org/fr/2.7/library/os.html)
 : contient diverses fonctions pour gérer l'os

* [pandas](https://pandas.pydata.org/pandas-docs/version/0.25/) : python library for data analysis

* [re](https://docs.python.org/3/library/re.html) : python library for regular expressions

* [nltk](https://www.nltk.org/) : python library to do things like   tokenization, stemming, ..

In [0]:
import tweepy 
import textblob
import os
import pandas
import re
import nltk


We're going to use [OAuthHandler](http://docs.tweepy.org/en/v3.5.0/auth_tutorial.html) to deal with authentification :


In [0]:
from tweepy import OAuthHandler 
from textblob import TextBlob 

We define a classe Twitter client :

In [43]:
class TwitterClient(object): 
	''' 
	Generic Twitter Class for sentiment analysis. 
	'''
	def __init__(self): 
		''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console 
		consumer_key = 'MlbpfXnAwJeQzQlRYK0dKJ6nB'
		consumer_secret = 'cTZLfoM7xOHpFjSfzUW0Ff0AsKPNsqEOhjMNiMbNW6Eg0NZpWr'
		access_token = '1179886074614689793-VLHbcHpSJoXkml9wVMqd9gIcSs4zMS'
		access_token_secret = 'Vnpf7cZp2uIUsdDDV6egSHdxDQXyCHf7INdKVa8Uqs2H4'

		# attempt authentication 
		try: 
			# create OAuthHandler object 
			self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
			self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
			self.api = tweepy.API(self.auth) 
		except: 
			print("Error: Authentication Failed") 

	def clean_tweet(self, tweet): 
		''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

	def get_tweet_sentiment(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return 'positive'
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return 'negative'

	def get_tweets(self, query, count = 10): 
		''' 
		Main function to fetch tweets and parse them. 
		'''
		# empty list to store parsed tweets 
		tweets = [] 

		try: 
			# call twitter api to fetch tweets 
			fetched_tweets = self.api.search(q = query, count = count) 

			# parsing tweets one by one 
			for tweet in fetched_tweets: 
				# empty dictionary to store required params of a tweet 
				parsed_tweet = {} 

				# saving text of tweet 
				parsed_tweet['text'] = tweet.text 
				# saving sentiment of tweet 
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

				# appending parsed tweet to tweets list 
				if tweet.retweet_count > 0: 
					# if tweet has retweets, ensure that it is appended only once 
					if parsed_tweet not in tweets: 
						tweets.append(parsed_tweet) 
				else: 
					tweets.append(parsed_tweet) 

			# return parsed tweets 
			return tweets 

		except tweepy.TweepError as e: 
			# print error (if any) 
			print("Error : " + str(e)) 


Positive tweets percentage: 39.130434782608695 %
Negative tweets percentage: 14.492753623188406 %


Positive tweets:
RT @joshuawongcf: Last week, the #HongKong Government invoked the Emergency Regulations Ordinance to outlaw face masks, curbing our right t…
RT @Jay_Watt: Harry Harrison #cartoon in today's @SCMPNews pointing out the new normal for the #HongKong police: they absolutely will deman…
RT @Stand_with_HK: How can the youth not be worried?

We only see a breakdown in leadership, literally everywhere.

And no one is brave eno…
RT @mexsystem28: Imagine making a perfect representation of what the hongkong people are going through but not be willing to stand with the…
RT @sankakucomplex: Top trending from Wednesday: Blizzard Banning Players Who Support Hong Kong - https://t.co/Z9viaE2yOz #Blizzard #HongKo…
RT @German_Hermit: This is amazing 

Reddit changed Blizzards promo Trailer for Overwatch, to make the Chinese Character Support Hongkong,…
RT @9GAG: Blizzard: Not us
#HongKong 


In [0]:

def main(query): 
	# creating object of TwitterClient Class 
	api = TwitterClient() 
	# calling function to get tweets 
	tweets = api.get_tweets(query, count = 200) 

	# picking positive tweets from tweets 
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
	# percentage of positive tweets 
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
	# picking negative tweets from tweets 
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
	# percentage of negative tweets 
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
	# percentage of neutral tweets 
#	print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 

	# printing first 5 positive tweets 
	print("\n\nPositive tweets:") 
	for tweet in ptweets[:10]: 
		print(tweet['text']) 

	# printing first 5 negative tweets 
	print("\n\nNegative tweets:") 
	for tweet in ntweets[:10]: 
		print(tweet['text']) 


Analyse des tweets sur Hong kong

In [46]:

if __name__ == "__main__": 
	# calling main function 
	main('Hong kong') 


Positive tweets percentage: 37.80487804878049 %
Negative tweets percentage: 18.29268292682927 %


Positive tweets:
@OrangeGrove55 @v_jvee They banned a professional hearthstone player and took his winnings after he said “free Hong… https://t.co/PxASWBIBo2
RT @Slasher: in tonight's Collegiate Hearthstone championship, American University held up a "Free Hong Kong, boycott Blizzard" during the…
RT @gogowachs: Thanks to the @sixers for kicking me out of the game for supporting Hong Kong
RT @Yuumei_Art: Blizzard banned pro Hearthstone winner for supporting free Hong Kong and took away his prize money. It would be SUCH A SHAM…
FREE HONG KONG.

I would like to start a crusade to take China but I know it won't go well
RT @CompoundBoss: With all this news about the #HongKongProstests I feel I need to tell people something I heard many yrs ago.
Apparently,…
RT @annafifield: The Hong Kong Protests are Also a Fight for a Free Press: 
“Unlike most of the protesters, we’re looking to be in the wron